<a href="https://colab.research.google.com/github/giolotar/Turkiye_detect/blob/main/Turkiye_Detect_Model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Rapid response Mapping of Rubble Detection in the cities of Antakya and Kahramanmaraş, Turkiye**

### The following Project utilizes Mask RCNN for Rubble detection following the 7.8 Magnitude earthquake that struck the Gazantiep Region of Turkey.

The cities of Antakya and Kharamanmaras have been majorly hit by the earthquake.

The project utilises the detectron2 library to construct the model and run inference. Two very high resolution satellite imagery have been selected for the city of Antakya and Kharamanmaras.

The image from the city of Antakya has been selected to conduct training and validation of the model, while the city of Kharamanmaras to run testing and model generalization.

The image has been tiled into 1024x1024 patches, COCO-format annotations have been created on CVAT. This is a relatively small dataset aiming to run rapid inference on the status of rubble

The products are a shapefile which can be utilized to generate a webMap or for further analysis by emergency response Agencies

# **SET-UP**

The set up includes installing the dependencies and importing the required Detectron2 Libraries.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Install Detectron2 Library
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

#Install geospatial libraries
!pip install rasterio fiona shapely

In [ ]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
import numpy as np
import os, json, cv2, random, math, re
from google.colab.patches import cv2_imshow

#import geospatial libraries
from shapely.geometry import Polygon, mapping
import fiona
import rasterio
from rasterio.enums import ColorInterp
import glob

# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances

# **Custom Dataset Creation**

To run inference for instance segmentation, the model requires COCO-annotations in a JSON format.

There are multiple online softwares for making annotations; for this exercise CVAT has been used https://app.cvat.ai/.

Detectron2 makes it very easy to register instances with few lines of code.

In [ ]:

#Here we register the coco-annotations (instances) for both the train dataset and validation dataset providing the path to the json files and the folders with the respective images
register_coco_instances("dataset_train", {}, "/content/drive/MyDrive/colabDL/tile1/train_ann/train_ann.json", "/content/drive/MyDrive/colabDL/tile1/train_row6_7")
register_coco_instances("dataset_val", {}, "/content/drive/MyDrive/colabDL/tile1/val_ann/val_ann.json", "/content/drive/MyDrive/colabDL/tile1/Val_row9")

In [ ]:
train_metadata = MetadataCatalog.get("dataset_train")
train_dataset_dicts = DatasetCatalog.get("dataset_train")

In [ ]:
val_metadata = MetadataCatalog.get("dataset_val")
val_dataset_dicts = DatasetCatalog.get("dataset_val")

## Let's Visualize some random Samples from our Validation and Train Dataset

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# Visualize some random samples from the train dataset
for d in random.sample(val_dataset_dicts, 2):
    img = cv2.imread(d["file_name"])

    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.show()

In [ ]:
# Visualize some random samples from the train dataset
for d in random.sample(train_dataset_dicts, 2):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.show()

# **TRAINING**

In the following lines of code we can begin training our Model.

First we have to create a configuration file to build the Hyperparameters of our MRCNN Model

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.modeling import build_model

In [ ]:
#Begin by creating a configuration variable cfg

cfg = get_cfg()
cfg.OUTPUT_DIR = "/content" #path where the configuration file .yaml will be saved

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml")) #this is a preconfigured Mask RCNN model from the Model Zoo of Detectron2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml") #pre-trained weights to kickstart model training.

cfg.DATASETS.TRAIN = ("dataset_train",) #specifies the training dataset to use
cfg.DATASETS.TEST = () #this is to add a test dataset. ! in this case no test dataset is being used for it is a rapid model inference building
cfg.DATALOADER.NUM_WORKERS = 2

cfg.SOLVER.IMS_PER_BATCH = 4  # refers to the number of images included in each batch that the model's optimizer processes during one training step
cfg.SOLVER.BASE_LR = 0.005  # How to pick a good learning rate? Trick = BASE_LR = 0.02 × (IMS_PER_BATCH / 16)
cfg.SOLVER.SCHEDULER_NAME = "WarmupMultiStepLR" #cfg.SOLVER.SCHEDULER_NAME = "WarmupCosineLR"

#Scheduling Iterations
cfg.SOLVER.MAX_ITER = 1000 #number of iterations of the model. To adjust based on computing power and dataset size
cfg.SOLVER.STEPS = (600, 800)     # decay at 60% and 80% of all the iterations
cfg.SOLVER.GAMMA = 0.1

# WARMUP iterations allow for intial model training
cfg.SOLVER.WARMUP_METHOD = "linear"
cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 10000

#anchors for small buildings
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16], [32], [64], [128], [256]] #Sets the anchor box scales
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.5, 0.75, 1.0, 1.33, 2.0]] #width:height ratios for anchors at every FPN level.
cfg.MODEL.RPN.PRE_NMS_TOPK_TRAIN = 4000
cfg.MODEL.RPN.PRE_NMS_TOPK_TEST  = 4000
cfg.MODEL.RPN.POST_NMS_TOPK_TRAIN = 2000
cfg.MODEL.RPN.POST_NMS_TOPK_TEST  = 2000
cfg.MODEL.RPN.NMS_THRESH = 0.6

# Correcting IN_FEATURES to match the backbone output before FPN
cfg.MODEL.FPN.IN_FEATURES = ["res2", "res3", "res4", "res5"]

#this resizes the tiles even further; configuration set for tiles that are 1024x1024
cfg.INPUT.MIN_SIZE_TRAIN = (1024, 1280, 1400)
cfg.INPUT.MIN_SIZE_TRAIN_SAMPLING = "choice"
cfg.INPUT.MAX_SIZE_TRAIN = 1800
cfg.INPUT.MIN_SIZE_TEST = 1400
cfg.INPUT.MAX_SIZE_TEST = 1800

#ROI Sampling

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024
cfg.MODEL.ROI_HEADS.POSITIVE_FRACTION = 0.5
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.4
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True


# Quality-of-life
cfg.SOLVER.AMP.ENABLED = True
cfg.TEST.EVAL_PERIOD = 100           # evaluates every 1000 iters
cfg.SOLVER.CHECKPOINT_PERIOD = 100   # Saves the model every 1000 iterations

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) #Create an instance of of DefaultTrainer with the given congiguration
trainer.resume_or_load(resume=False) #Load a pretrained model if available (resume training) or start training from scratch if no pretrained model is available

In [ ]:
trainer.train() #Start the training process

In [ ]:
import yaml
# Save the configuration to a config.yaml file
config_yaml_path = ""
with open(config_yaml_path, 'w') as file:
    yaml.dump(cfg, file)

# **Inference on Test Images**

We run inference by generating a prediction setting the testing thershold

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "/content/model_finale.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set a custom testing threshold
predictor = DefaultPredictor(cfg) #predictor draws from the yaml configuration

## EVALUATION **Check** average precision and recall

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "dataset_val")
print(inference_on_dataset(predictor.model, val_loader, evaluator))


m,ore evaluation metrics for the model

# **Inference on Out of Domain Images**

Then we run the predictor on an unseen Georeferenced TIF image, the image gets tiled, predictions are run on the tiles and exported in .npz as georeferenced polygons.
Then they are turned into a shapefile

### Generate Shapefile with predicted instances as Georeferenced Polygons

In [ ]:
# ==== USER PATHS ====
INPUT_TIFF = "/content/drive/MyDrive/colabDL/Turkiye/TR1/1_GEOTIFF.tif" # Use the georeferenced TIFF
OUT_DIR   = "/content/drive/MyDrive/colabDL/Turkiye/TR1"  # main output directory
NPZ_OUT_DIR = os.path.join(OUT_DIR, "npz_preds") # subdirectory for npz files
TILE = 1024

os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(NPZ_OUT_DIR, exist_ok=True)

# Read the georeferenced image and get georeferencing info
try:
    with rasterio.open(INPUT_TIFF) as src:
        img = src.read()
        # Rasterio reads bands first (C, H, W), convert to OpenCV format (H, W, C)
        img = np.transpose(img, (1, 2, 0))
        # Explicitly select the first 3 channels to handle potential alpha channels
        if img.shape[2] == 4:
            img = img[:, :, :3]
        orig_h, orig_w = img.shape[:2]
        transform = src.transform # Affine transform for georeferencing
        crs = src.crs # Coordinate reference system

except rasterio.errors.RasterioIOError as e:
    raise FileNotFoundError(f"Could not read georeferenced image at {INPUT_TIFF}: {e}")

# Pad to multiples of TILE so we don’t lose right/bottom edges
H = math.ceil(orig_h / TILE) * TILE
W = math.ceil(orig_w / TILE) * TILE
pad_bottom = H - orig_h
pad_right  = W - orig_w

if pad_bottom or pad_right:
    # Use the last pixel value for padding to avoid edge effects if possible,
    # otherwise use black padding for color images.
    if img.shape[2] == 3:
        pad_val = [int(img[-1,-1,0]), int(img[-1,-1,1]), int(img[-1,-1,2])]
    else:
        pad_val = [0] * img.shape[2]
    img = cv2.copyMakeBorder(
        img, 0, pad_bottom, 0, pad_right,
        borderType=cv2.BORDER_CONSTANT, value=pad_val
    )

rows = H // TILE
cols = W // TILE

tiles_written = 0
dets_saved = 0

for y in range(rows):
    for x in range(cols):
        r0 = y * TILE
        c0 = x * TILE
        tile = img[r0:r0+TILE, c0:c0+TILE]

        # Calculate georeferenced origin of the tile (top-left corner)
        geo_x0, geo_y0 = transform * (c0, r0)

        # ---- Run inference on the tile
        outputs = predictor(tile)
        inst = outputs["instances"].to("cpu")

        # ---- Extract raw predictions
        boxes  = inst.pred_boxes.tensor.numpy() if inst.has("pred_boxes") else np.zeros((0,4), dtype=np.float32)
        scores = inst.scores.numpy()             if inst.has("scores")     else np.zeros((0,), dtype=np.float32)
        classes= inst.pred_classes.numpy()       if inst.has("pred_classes") else np.zeros((0,), dtype=np.int32)
        masks  = inst.pred_masks.numpy()         if inst.has("pred_masks") else None  # (N, H, W) boolean

        # ---- SAVE RAW PREDICTIONS (compressed)
        # Include tile origin in pixel and georeferenced coordinates
        npz_path = os.path.join(NPZ_OUT_DIR, f"tile_{y}_{x}_pred.npz")
        np.savez_compressed(
            npz_path,
            boxes=boxes,
            scores=scores,
            classes=classes,
            masks=masks,                 # None is allowed; np.load(..., allow_pickle=True) will handle it
            tile_row=y,
            tile_col=x,
            tile_origin_rc=np.array([r0, c0], dtype=np.int32), # Pixel origin (row, col)
            tile_origin_geo=np.array([geo_x0, geo_y0], dtype=np.float64), # Georeferenced origin (x, y)
            tile_size=TILE,
            padded_h=H,
            padded_w=W,
            orig_h=orig_h,
            orig_w=orig_w,
            pad_bottom=pad_bottom,
            pad_right=pad_right
        )
        dets_saved += len(boxes)

print(f"Done. Wrote {tiles_written} visualized tiles to {OUT_DIR}")
print(f"Saved predictions for {rows*cols} tiles ({dets_saved} detections total) as .npz files to {NPZ_OUT_DIR}")

Convert from the npz to shp retaining geographic coordinates

In [ ]:
# Directory containing the predicted tile .npz files
PREDICTED_NPZ_DIR = "/content/drive/MyDrive/colabDL/Turkiye/TR1/npz_preds" # Assuming npz files are in a subdirectory

OUTPUT_VECTOR_PATH = "/content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_1.gpkg"

ORIGINAL_GEOREFERENCED_IMAGE_PATH = "/content/drive/MyDrive/colabDL/Turkiye/TR1/1_GEOTIFF.tif" # Example path, replace with your actual georeferenced image path

# --- Function to get georeferencing information ---
def get_georeferencing_info(image_path):
    """Reads georeferencing information (transform and CRS) from a georeferenced image."""
    try:
        with rasterio.open(image_path) as src:
            return src.transform, src.crs
    except rasterio.errors.RasterioIOError as e:
        print(f"Error reading georeferencing info from {image_path}: {e}")
        print("Assuming pixel coordinates. The output vector file will NOT be georeferenced.")
        return None, None

# Get georeferencing info from the original image
transform, crs = get_georeferencing_info(ORIGINAL_GEOREFERENCED_IMAGE_PATH)

# --- Define the schema for the vector file ---

schema = {
    'geometry': 'Polygon',
    'properties': {'score': 'float', 'class_id': 'int'},
}

# --- Create the vector file (Geopackage) ---
driver = 'GPKG'

with fiona.open(
    OUTPUT_VECTOR_PATH,
    'w',
    driver,
    schema,
    crs=crs # Use the obtained CRS, or None if not available
) as collection:
    # List all .npz files in the directory
    npz_filenames = [f for f in os.listdir(PREDICTED_NPZ_DIR) if f.endswith('_pred.npz')]

    if not npz_filenames:
        print("No predicted .npz files found in the directory.")
    else:
        # Iterate through each .npz file
        for filename in npz_filenames:
            npz_path = os.path.join(PREDICTED_NPZ_DIR, filename)
            with np.load(npz_path, allow_pickle=True) as data:
                masks = data['masks']
                scores = data['scores']
                classes = data['classes']
                tile_origin_rc = data['tile_origin_rc'] # Pixel origin (r0, c0)


                if masks is not None and masks.shape[0] > 0:
                    # Process each instance mask within the tile
                    for i in range(masks.shape[0]):
                        instance_mask = masks[i]
                        score = scores[i]
                        class_id = classes[i]


                        mask_uint8 = instance_mask.astype(np.uint8) * 255
                        # Use cv2.RETR_TREE and cv2.CHAIN_APPROX_NONE for more detailed contours if needed,
                        # but SIMPLE is usually sufficient for polygons.
                        contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                        # Convert contours to polygons and transform coordinates
                        for contour in contours:

                            contour_points_tile = contour.squeeze().tolist()

                            if not isinstance(contour_points_tile, list):
                                # Handle cases where contour.squeeze() results in a single point
                                contour_points_tile = [contour_points_tile]
                            elif not all(isinstance(p, list) for p in contour_points_tile):
                                # Handle cases where contour.squeeze() results in a flat list of coords
                                contour_points_tile = [contour_points_tile]


                            if len(contour_points_tile) < 3:
                                continue # Skip if not enough points to form a polygon

                            # Convert tile pixel coordinates to padded image pixel coordinates
                            # Tile origin is (row, col), contour points are (col, row)
                            contour_points_padded = [(p[0] + tile_origin_rc[1], p[1] + tile_origin_rc[0]) for p in contour_points_tile]

                            # Convert padded image pixel coordinates to geographic coordinates using the original image's transform
                            if transform:
                                # Apply the affine transform to each point
                                # transform * (col, row) gives (x, y) georeferenced coordinates
                                contour_points_geo = [transform * (p[0], p[1]) for p in contour_points_padded]
                                polygon = Polygon(contour_points_geo)
                            else:
                                # If no transform, use padded pixel coordinates for the polygon
                                polygon = Polygon(contour_points_padded)




                                # Add the polygon and its properties to the vector file
                            if not polygon.is_empty and polygon.exterior: # Ensure polygon is valid and not empty
                                # Cast score to a standard float type to potentially avoid Fiona warning
                                collection.write({
                                    'geometry': mapping(polygon),
                                    'properties': {'score': float(score), 'class_id': int(class_id)},
                                })

print(f"Vector file (Geopackage) saved to {OUTPUT_VECTOR_PATH}")

### Produce a .TIF image mosaic with the predictions and bounding boxes printed on the image.

In [ ]:
#use npz to create a geotiff

# ==== USER PATHS ====
INPUT_TIFF = "/content/drive/MyDrive/colabDL/Turkiye/TR1/1_GEOTIFF.tif"  # original georeferenced image
OUT_DIR    = "/content/drive/MyDrive/colabDL/Turkiye/TR1/Rubble_2_tif"   # output directory
OUTPUT_GEOTIFF = os.path.join(OUT_DIR, "Rubble_1_predictions_vis.tif")   # final mosaic GeoTIFF
PREDICTED_NPZ_DIR = "/content/drive/MyDrive/colabDL/Turkiye/TR1/npz_preds"  # folder with *_pred.npz
TILE = 1024

os.makedirs(OUT_DIR, exist_ok=True)

# ---------- helpers ----------
def color_from_class_id(class_id: int) -> tuple:

    rng = np.random.RandomState(class_id * 1337 + 42)
    return tuple(int(x) for x in rng.choice(256, size=3))  # BGR

def put_label(img, text, pt, bg=(0,0,0), fg=(255,255,255)):

    font = cv2.FONT_HERSHEY_SIMPLEX
    scale = 0.5
    thickness = 1
    (tw, th), baseline = cv2.getTextSize(text, font, scale, thickness)
    x, y = pt
    cv2.rectangle(img, (x, y - th - baseline), (x + tw, y + baseline), bg, -1, cv2.LINE_AA)
    cv2.putText(img, text, (x, y), font, scale, fg, thickness, cv2.LINE_AA)

# ---------- visualization config (ADD THIS) ----------
USE_FIXED_COLOR = True                # force one color for all instances
FIXED_COLOR_BGR = (0, 0, 255)         # red in BGR
ALPHA = 0.45                          # mask fill opacity
DRAW_BBOX = True                      # draw bounding boxes
LABEL_OFFSET = (2, 12)                # (x,y) offset inside top-left corner of bbox for the label

# ---------- read base imagery & georef ----------
try:
    with rasterio.open(INPUT_TIFF) as src:
        base = src.read()                # (C,H,W)
        transform = src.transform
        crs = src.crs
except rasterio.errors.RasterioIOError as e:
    raise FileNotFoundError(f"Could not read georeferenced image at {INPUT_TIFF}: {e}")

# Convert to HWC (OpenCV) and force 3 channels for visualization
base = np.transpose(base, (1, 2, 0))     # (H,W,C)
if base.shape[2] >= 3:
    base_rgb = base[:, :, :3]
else:
    base_rgb = np.repeat(base, 3, axis=2)

orig_h, orig_w = base_rgb.shape[:2]

# ---------- pad canvas to tile grid (optional but handy) ----------
H = math.ceil(orig_h / TILE) * TILE
W = math.ceil(orig_w / TILE) * TILE
pad_bottom = H - orig_h
pad_right  = W - orig_w

if pad_bottom > 0 or pad_right > 0:
    pad_val = [int(base_rgb[-1,-1,0]), int(base_rgb[-1,-1,1]), int(base_rgb[-1,-1,2])]
    canvas = cv2.copyMakeBorder(base_rgb, 0, pad_bottom, 0, pad_right,
                                borderType=cv2.BORDER_CONSTANT, value=pad_val)
else:
    canvas = base_rgb.copy()

# We will draw directly onto canvas (BGR for OpenCV convenience)
canvas = canvas[:, :, ::-1]  # convert to BGR

# ---------- iterate over npz tiles and draw ----------
npz_files = sorted([f for f in os.listdir(PREDICTED_NPZ_DIR) if f.endswith("_pred.npz")])
if not npz_files:
    print(f"No *_pred.npz files found in {PREDICTED_NPZ_DIR}")

instances_drawn = 0

for fname in npz_files:
    fpath = os.path.join(PREDICTED_NPZ_DIR, fname)
    try:
        with np.load(fpath, allow_pickle=True) as data:
            masks = data.get('masks', None)        # (N, th, tw) boolean/0-1
            scores = data.get('scores', None)      # (N,)
            classes = data.get('classes', None)    # (N,)
            tile_origin_rc = data.get('tile_origin_rc', None)  # (r0, c0)
    except Exception as e:
        print(f"Skipping {fname}: cannot read npz ({e})")
        continue

    if masks is None or masks.size == 0:
        continue
    if tile_origin_rc is None:

        try:
            base_name = os.path.splitext(fname)[0]
            parts = base_name.split('_')
            r0 = int([p for p in parts if p.startswith('r')][0][1:])
            c0 = int([p for p in parts if p.startswith('c')][0][1:])
        except Exception:
            print(f"Skipping {fname}: missing tile_origin_rc and filename not parseable.")
            continue
    else:
        r0, c0 = int(tile_origin_rc[0]), int(tile_origin_rc[1])

    N, th, tw = masks.shape
    # Bounds on canvas
    r1 = r0 + th
    c1 = c0 + tw
    if r0 >= canvas.shape[0] or c0 >= canvas.shape[1]:
        continue  # tile completely outside padded canvas
    # Clip to canvas in case of edge/padding mismatch
    rr0, cc0 = max(0, r0), max(0, c0)
    rr1, cc1 = min(canvas.shape[0], r1), min(canvas.shape[1], c1)
    if rr1 <= rr0 or cc1 <= cc0:
        continue

    # Region of interest on canvas (make a contiguous copy for drawing)
    roi = canvas[rr0:rr1, cc0:cc1].copy()  # Make a copy here

    # Matching region in tile coordinates
    tr0 = rr0 - r0
    tc0 = cc0 - c0
    tr1 = th - (r1 - rr1)
    tc1 = tw - (c1 - cc1)

    # Draw each instance
    for i in range(N):
        m = masks[i, tr0:tr1, tc0:tc1].astype(np.uint8)  # (h,w) 0/1
        if m.max() == 0:
            continue
        score = float(scores[i]) if scores is not None else None
        class_id = int(classes[i]) if classes is not None else 0

        bgr = FIXED_COLOR_BGR if USE_FIXED_COLOR else color_from_class_id(class_id)

        alpha = ALPHA

        # Overlay fill (mask tint)
        colored = np.zeros_like(roi, dtype=np.uint8)
        colored[:] = bgr
        mask3 = np.repeat(m[:, :, None], 3, axis=2)  # (h,w,3)
        roi[:] = np.where(mask3 == 1, (alpha * colored + (1 - alpha) * roi).astype(np.uint8), roi)

        # Contour outline (optional; keeping it helps visually)
        contours, _ = cv2.findContours((m * 255), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(roi, contours, -1, bgr, 2, cv2.LINE_AA)

        # ---- draw bounding box from the mask ----
        if DRAW_BBOX:
            ys, xs = np.where(m > 0)
            if xs.size and ys.size:
                x0, y0 = int(xs.min()), int(ys.min())
                x1, y1 = int(xs.max()), int(ys.max())
                cv2.rectangle(roi, (x0, y0), (x1, y1), bgr, 2, cv2.LINE_AA)

        # ---- label at top-left of bbox ----
        if score is not None:
            pct = int(round(score * 100)) if 0.0 <= score <= 1.0 else int(round(score))
            label = f"{pct}%"
        else:
            label = ""

        if label:
            # If we have a bbox, anchor to its top-left; otherwise, fallback to contour’s top-left
            if DRAW_BBOX and xs.size and ys.size:
                x0_lbl, y0_lbl = x0, y0
            else:
                # Fallback: use contour’s top-left from the largest contour (by area)
                if contours:
                    areas = [cv2.contourArea(c) for c in contours]
                    k = int(np.argmax(areas))
                    x0_lbl, y0_lbl, w_lbl, h_lbl = cv2.boundingRect(contours[k])
                else:
                    x0_lbl, y0_lbl = 0, 0

            # Apply an offset so text is *inside* the region
            ox, oy = LABEL_OFFSET
            lx = max(0, min(x0_lbl + ox, roi.shape[1] - 1))
            ly = max(12, min(y0_lbl + oy, roi.shape[0] - 1))
            put_label(roi, label, (lx, ly), bg=(0, 0, 0), fg=(255, 255, 255))

        instances_drawn += 1

    canvas[rr0:rr1, cc0:cc1] = roi


# ---------- crop back and write geotiff ----------
canvas = canvas[:, :, ::-1]  # back to RGB
mosaic_vis_cropped = canvas[:orig_h, :orig_w, :]

profile = {
    "driver": "GTiff",
    "height": orig_h,
    "width":  orig_w,
    "count":  3,
    "dtype":  "uint8",
    "crs":    crs,
    "transform": transform,
    "compress": "lzw",
    "photometric": "RGB",
    "interleave": "pixel",
}

with rasterio.open(OUTPUT_GEOTIFF, "w", **profile) as dst:
    dst.write(np.transpose(mosaic_vis_cropped, (2, 0, 1)))
    dst.colorinterp = (ColorInterp.red, ColorInterp.green, ColorInterp.blue)

print(f"Done. Drew {instances_drawn} instances from {len(npz_files)} tiles.")
print(f"Wrote mosaic GeoTIFF with printed predictions to:\n{OUTPUT_GEOTIFF}")

Visualize the Image with the predicted instances

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

image_to_display_path = OUTPUT_GEOTIFF

image_to_display = cv2.imread(image_to_display_path, -1)

if image_to_display is None:
    print(f"Error: Could not read image at {image_to_display_path}. Please check the file path.")
else:

    print(f"Displaying image from: {image_to_display_path}")
    cv2_imshow(image_to_display)